**Network architecture**
- [ ] Number of hidden layers (network depth) 
- [ ] Number of neurons in each layer (layer width) 
- [ ] Activation type 

**Learning and optimization**
- [ ] Learning rate and decay schedule
- [ ] Mini-batch size
- [ ] Optimization algorithms
- [ ] Number of training iterations or epochs (and early stopping criteria)

**Regularization techniques to avoid overfitting** 
- [ ] L2 regularization
- [ ] Dropout layers
- [ ] Data augmentation
- [ ] Batch normalization
- [ ] Transfer learning


# 變數
- [ ] load_weights : val_loss: 0.4617 - val_accuracy: 0.8414
- [ ] target size : 450*450
- [ ] learnging rate: 1e-6
- [ ] batch size : 10
- [ ] class weight



# 結果
- [ ] val_loss: 0.4693 - val_accuracy: 0.8495


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# select 'add to my drive' on the shared folder

root_path = 'gdrive/MyDrive/final_project'
train_dir='gdrive/MyDrive/final_project/train/train' #train jpg save place 
train_df = pd.read_csv(f'{root_path}/train_data.csv')


train_df=train_df.append(train_df.loc[1,:],ignore_index=True)
train_df['Type'] = train_df['Type'].astype('str') #如果class_mode = 'sparse'要是string要是string

## InceptionResNetV2

In [ ]:
# Data agumentation and pre-processing using tensorflow
gen = ImageDataGenerator(
                  rescale=1./255.,
                  horizontal_flip = True,
                  validation_split=0.2005 # training: 80% data, validation: 20% data
                 )
  
train_generator = gen.flow_from_dataframe(
    train_df, # dataframe
    directory = train_dir, # images data path / folder in which images are there
    x_col = 'Name',
    y_col = 'Type',
    subset="training",
    color_mode="rgb",
    target_size = (600,600), # image height , image width
    class_mode="categorical",
    batch_size=10,
    shuffle=True,
    seed=42,
)
  
  
validation_generator = gen.flow_from_dataframe(
    train_df, # dataframe
    directory = train_dir, # images data path / folder in which images are there
    x_col = 'Name',
    y_col = 'Type',
    subset="validation",
    color_mode="rgb",
    target_size = (600,600), # image height , image width
    class_mode="categorical",
    batch_size=10,
    shuffle=True,
    seed=42,
)

Found 7420 validated image filenames belonging to 4 classes.
Found 1860 validated image filenames belonging to 4 classes.


In [ ]:
# train_labels=np.array([])
# for i in tqdm(range(train_generator.n//train_generator.batch_size)):
#   feat = train_generator[i][-1]
#   labels = np.argmax(feat, axis=1)
#   train_labels = np.append(train_labels,labels)


In [ ]:
# from sklearn.utils import class_weight
# ClassWeights = dict(zip(np.unique(train_labels),
#                         class_weight.compute_class_weight('balanced',
#                                                 classes=np.unique(train_labels),y=train_labels)))
# print(ClassWeights) 

In [ ]:
ClassWeights ={0.0: 0.8672276764843385, 1.0: 1.3081805359661496, 2.0: 1.1906290115532734, 3.0: 0.8047722342733189}

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping


# load the InceptionResNetV2 architecture with imagenet weights as base
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(600,600,3)
                     )
  
base_model.trainable=False
# For freezing the layer we make use of layer.trainable = False
# means that its internal state will not change during training.
# model's trainable weights will not be updated during fit(),
# and also its state updates will not run.
  
model = tf.keras.Sequential([ 
        base_model,   
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

opt = optimizers.Adam(learning_rate = 0.0001)

model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])
# categorical cross entropy is taken since its used as a loss function for 
# multi-class classification problems where there are two or more output labels.
# using Adam optimizer for better performance
# other optimizers such as sgd can also be used depending upon the model

mc = ModelCheckpoint(os.path.join('gdrive/MyDrive/final_project/code/resnetV2_600_mod_dahei_unlabeltest.h5'),
                     monitor='val_accuracy',
                     verbose=1,
                     save_best_only=True,
                     save_weights_only=True,
                     mode='max')

early = tf.keras.callbacks.EarlyStopping(  monitor = 'val_accuracy',
                       mode = 'max',
                       patience=6,
                       min_delta=0.0001,
                       restore_best_weights=True)



batch_size=10
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size
  


219070464/219055592 [==============================] - 2s 0us/step


In [ ]:
base_model.trainable = True
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),loss='categorical_crossentropy',metrics=['accuracy'])

# history = model.fit(
#                     train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=validation_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     class_weight = ClassWeights,
#                     epochs=50,
#                     callbacks=[early,mc]
#                     )

model.load_weights('gdrive/MyDrive/final_project/weights/resnetV2_600_unlabel_mod_3_interuptted.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 17, 17, 1536)     54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 17, 17, 1536)     10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               196736    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        

In [ ]:
image_folder = 'gdrive/MyDrive/final_project/unlabeled_data/'
image_list = os.listdir(image_folder)
image_list.remove('oIQiOVTSKZ9741837007.jpg')
classifier_results = []
for image in range(len(image_list)):
#for image in range(1):
    test_image = tf.keras.preprocessing.image.load_img(image_folder + image_list[image], target_size = (600, 600))
    test_image = tf.keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = test_image/255
    result = model.predict(test_image)
    if result.max()>0.85:
      result_arg = np.argmax(result)
    else:
      result_arg = np.nan
    classifier_results.append(result_arg)

In [ ]:
classifier_results

[nan]

In [ ]:
train_df = pd.read_csv(f'{root_path}/train_data.csv')
pseudo_df = pd.DataFrame(list(zip(image_list, classifier_results)), columns=['Name', 'Type'])
pseudo_df.shape

(5276, 2)

In [ ]:
pseudo_df = pseudo_df.dropna()
pseudo_df.shape

(4894, 2)

In [ ]:
new_train_df = train_df.append(pseudo_df)
new_train_df.shape

(14173, 2)

In [ ]:
new_train_df.to_csv('gdrive/MyDrive/final_project/train_and_unlabeled/600_unlabel_4.csv', index=False)